In [38]:
import numpy as np
import pandas as pd

In [39]:
input_file = "20210311_v0.2-FSI" # do not put file extension

df = pd.read_csv("{}.csv".format(input_file))

df.describe()

,Entity Registration Date,Entity Profile UEN,Entity Name,Primary Section Description
count,6389,6389,6389,6389
unique,342,6197,6382,1
top,2020-03-13,202036310W,ALL STAR CAPITAL PTE. LTD.,FINANCIAL AND INSURANCE ACTIVITIES
freq,60,3,2,6389


In [40]:
df

,Entity Registration Date,Entity Profile UEN,Entity Name,Primary Section Description
0,2020-01-01,202000015R,JULIAN GREY PTE. LTD.,FINANCIAL AND INSURANCE ACTIVITIES
1,2020-01-01,202000015R,JULIAN GREY VENTURES PTE. LTD.,FINANCIAL AND INSURANCE ACTIVITIES
2,2020-01-01,202000017W,PRECIOUS (GLOBAL) PTE. LTD.,FINANCIAL AND INSURANCE ACTIVITIES
3,2020-01-01,202000018H,ALLIED STAR PTE. LTD.,FINANCIAL AND INSURANCE ACTIVITIES
4,2020-01-01,202000024W,LAUNCH I/O PTE. LTD.,FINANCIAL AND INSURANCE ACTIVITIES
...,...,...,...,...
6384,2020-12-17,T20VC0183A,SEAVI ADVENT EQUITY VII FUND VCC,FINANCIAL AND INSURANCE ACTIVITIES
6385,2020-12-23,T20VC0185D,WELLINGTON MANAGEMENT FUNDS (SINGAPORE) VCC,FINANCIAL AND INSURANCE ACTIVITIES
6386,2020-12-23,T20VC0187G,PENCO CAPITAL VCC,FINANCIAL AND INSURANCE ACTIVITIES
6387,2020-12-23,T20VC0190G,RAINMAKING VENTURES (S) VCC,FINANCIAL AND INSURANCE ACTIVITIES


In [41]:
import requests
import bs4
import re
from urllib.parse import urlparse, parse_qs
import time

# setup User Agent headers
headers = requests.utils.default_headers()
headers.update({
    'User-Agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/89.0.4389.82 Safari/537.36',
})

url = "https://sgpgrid.com/search-results?target={\"value\":\"Registration Number\",\"label\":\"Registration Number\",\"searchTarget\":\"registrationNumber\"}&value="

output_file = "{}-result.csv".format(input_file)
size = len(df) # size of df



# f.write("Entity Registration Date,Entity Profile UEN,Entity Name,Primary Section Description,Description,Website,Industry\n")

# Finding Description, Website, Industry
for i in range(0, 10):
    entity = df.loc[i] # current row record
    entity_uen = entity[1] # entity name
    entity_name = entity[2]
    
    
    print("Searching for id={} {} {}".format(i, entity_uen, entity_name))
    req = requests.get(url + entity_uen, headers=headers) # imitate Chrome's UA
    soup = bs4.BeautifulSoup(req.text, "html.parser")

    # Now look for elements
    columns = soup.find_all("div", {"class": "rt-td table-cell"})
    
    # we know that column 0 is description (sometimes same as SSIC), 3 is website, 5 is SSIC
    
    print(soup)

    
    description = columns[0].get_text()
    website = columns[3].get_text()

    ssic = columns[5].get_text()
    
    print(description, website, ssic)
    
    formatted_csv_row = "{},{},{},{}".format(
        entity[0],
        entity[1],
        entity[2],
        entity[3]
    )
    
    formatted_csv_row += ",\"{}\",\"{}\",\"{}\"".format(description, website, ssic)
    
    f = open(output_file, "a")
    f.write(formatted_csv_row + "\n")
    f.close()
    
    time.sleep(1)
    





Searching for id=0 202000015R JULIAN GREY PTE. LTD.
<!DOCTYPE html>
<html lang="en"><head><meta content="width=device-width, initial-scale=1, shrink-to-fit=no" name="viewport"/><meta charset="utf-8"/><meta content="IE=Edge" http-equiv="X-UA-Compatible"/><meta content="width=device-width, initial-scale=1, maximum-scale=1, user-scalable=no" name="viewport"/><meta content="no-cache, no-store, must-revalidate" http-equiv="Cache-Control"/><meta content="no-cache" http-equiv="Pragma"/><meta content="0" http-equiv="Expires"/><link href="/static/manifest.json" rel="manifest"/><link href="/static/favicon.svg" rel="shortcut icon" type="image/svg+xml"/><script async="" src="https://www.googletagmanager.com/gtag/js?id=UA-120017197-1" type="8d9f1387823faf3f7f2df50e-text/javascript"></script><script type="8d9f1387823faf3f7f2df50e-text/javascript">window.dataLayer = window.dataLayer || [];
                    function gtag(){dataLayer.push(arguments);}
                    gtag('js', new Date());
    

IndexError: list index out of range

In [ ]:

# f = open(output_file, "w")

# f.write("Entity Registration Date,Entity Profile UEN,Entity Name,Primary Section Description,Link 1,Link 2,Link 3,Link 4,Link 5,Link 6,Link 7,Link 8,Link 9,Link 10\n")

# for i in range(1, 20):
#     entity = df.loc[i] # current row record
#     entity_name = entity[2] # entity name
#     query = "{} singapore website".format(entity_name)
    
#     print("Searching for id={} \"{}\"".format(i, query))
    
#     # prepare to search "<entity name> singapore website" for more localised search context
#     req = requests.get(url.format(query))
#     soup = bs4.BeautifulSoup(req.text, "html.parser")
#     headers = soup.find_all("h3")
#     links = soup.find_all(href=re.compile(r'\/url\?q=')) # pick top 10 search results & its link
    
#     # file write here
    
#     formatted_csv_row = "{},{},{},{}".format(
#         entity[0],
#         entity[1],
#         entity[2],
#         entity[3]
#     )
    
#     for i in range(10):
#         google_url = links[i].get('href')
#         parsed_url = parse_qs(google_url)
#         formatted_csv_row += ",{}".format(parsed_url['/url?q'][0])
#     f.write(formatted_csv_row + "\n")
# f.close()


